In [1]:
import os
import pandas as pd

# Caminho do diretório onde os arquivos estão localizados
caminho_diretorio = '//172.31.8.209/Files/CataliseInvestimentos/14. BigData Catalise/Staging/PosicaoAtivo/'

# Lista todos os arquivos no diretório
arquivos = os.listdir(caminho_diretorio)

# Filtra apenas os arquivos CSV
arquivos_csv = [f for f in arquivos if f.endswith('.csv')]

if arquivos_csv:
    # Encontra o arquivo mais recente
    caminho_arquivo_csv = max([os.path.join(caminho_diretorio, f) for f in arquivos_csv], key=os.path.getmtime)

    # Lê o arquivo CSV mais recente
    df_csv_agis = pd.read_csv(caminho_arquivo_csv, delimiter=';', encoding='latin-1', dtype='str', index_col=False)
    
    # Seleciona apenas as colunas desejadas
    colunas_desejadas = ["CARTEIRA", "DATA", "SALDOCONTASPAGARRECEBER", "SALDOCAIXAATUAL", "VALORPOSICAOATIVOS"]
    Df_DM = df_csv_agis[colunas_desejadas]

    # Renomeia as colunas
    Df_DM.rename(columns={
        'CARTEIRA': 'NmFundo',
        'DATA': 'DtPosicao',
        'SALDOCONTASPAGARRECEBER': 'VlrReceber',
        'SALDOCAIXAATUAL': 'VlrCaixaAtual',
        'VALORPOSICAOATIVOS': 'VlrPosicaoAtivo'
    }, inplace=True)

    # Converte a coluna DtPosicao para o formato datetime
    Df_DM['DtPosicao'] = pd.to_datetime(Df_DM['DtPosicao'], format='%d/%m/%Y', errors='coerce')

    # Remove os parênteses e converte a coluna VlrReceber para números
    Df_DM['VlrReceber'] = Df_DM['VlrReceber'].str.replace('(', '-', regex=False)  # Substitui '(' por '-'
    Df_DM['VlrReceber'] = Df_DM['VlrReceber'].str.replace(')', '', regex=False)  # Remove ')'

    # Remove os pontos e substitui vírgulas por ponto
    Df_DM['VlrReceber'] = Df_DM['VlrReceber'].str.replace('.', '', regex=False)  # Remove pontos
    Df_DM['VlrReceber'] = Df_DM['VlrReceber'].str.replace(',', '.', regex=False)  # Substitui vírgula por ponto

    # Converte a coluna VlrReceber para float
    Df_DM['VlrReceber'] = pd.to_numeric(Df_DM['VlrReceber'], errors='coerce')

    # Para as colunas VlrCaixaAtual e VlrPosicaoAtivo, aplica as mesmas transformações
    for coluna in ['VlrCaixaAtual', 'VlrPosicaoAtivo']:
        Df_DM[coluna] = Df_DM[coluna].str.replace('.', '', regex=False)  # Remove pontos
        Df_DM[coluna] = Df_DM[coluna].str.replace(',', '.', regex=False)  # Substitui vírgula por ponto
        Df_DM[coluna] = pd.to_numeric(Df_DM[coluna], errors='coerce')  # Converte para float

else:
    print("Nenhum arquivo CSV encontrado no diretório.")

# Exibe o DataFrame
Df_DM.head()

C:\Users\atrindade\AppData\Local\Temp\2\ipykernel_4268\29823734.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Df_DM.rename(columns={
C:\Users\atrindade\AppData\Local\Temp\2\ipykernel_4268\29823734.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Df_DM['DtPosicao'] = pd.to_datetime(Df_DM['DtPosicao'], format='%d/%m/%Y', errors='coerce')
C:\Users\atrindade\AppData\Local\Temp\2\ipykernel_4268\29823734.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead



,NmFundo,DtPosicao,VlrReceber,VlrCaixaAtual,VlrPosicaoAtivo
0,FICFIM STAND BY,2025-03-20,-2210.53,2363.74,1.402270e+06
1,FICFIM AXXIA,2025-03-20,-9580.68,345.13,1.340230e+08
2,FICFIM BINVEST,2025-03-20,-5142.68,2908.69,1.786907e+07
3,FICFIM BELLAGIO,2025-03-20,-3157.90,1489.45,7.838899e+08
4,FIDC BINVEST SR,2025-03-20,0.00,0.00,4.467313e+07


In [2]:
import os
import pandas as pd
import mysql.connector

def AlimentaDM_Ativos(df):
    # Função para inserir um pedaço do DataFrame na tabela MySQL
    def insert_dataframe_to_mysql(df, connection, table_name):
        cursor = connection.cursor()
        for i, row in df.iterrows():
            # Substitui valores NaN por None para que MySQL aceite como NULL
            row = row.where(pd.notna(row), None)
            sql = f"INSERT INTO {table_name} ({', '.join(row.index)}) VALUES ({', '.join(['%s'] * len(row))})"
            cursor.execute(sql, tuple(row))
        connection.commit()
        cursor.close()

    # Conectar ao banco de dados MySQL
    connection = mysql.connector.connect(
        host='catalise-bi-dados-cluster.cluster-ciaao5zm9beh.sa-east-1.rds.amazonaws.com',
        database='DW_CORPORATIVO',
        user='admin',
        password='yb222KraY7PTN0jbH7P3'
    )

    if connection.is_connected():
        print('Conectado ao MySQL')

        # Dividir o DataFrame em pedaços menores para evitar problemas de memória
        tamanho_do_pedaço = 10000  # Ajuste conforme necessário
        total_registros = len(df)

        # Substitui NaN por None nas colunas do DataFrame
        df = df.where(pd.notna(df), None)  # Substitui todos os NaN do DataFrame por None

        for i in range(0, total_registros, tamanho_do_pedaço):
            pedaco_df = df.iloc[i:i + tamanho_do_pedaço]
            insert_dataframe_to_mysql(pedaco_df, connection, 'Dm_PosicaoAtivo')
            print(f"Registros inseridos: {i + len(pedaco_df)} / {total_registros}")

        print('Concluído')
    else:
        print('Falha na conexão ao MySQL')

    # Fechar a conexão com o banco de dados
    connection.close()

In [3]:
AlimentaDM_Ativos(Df_DM)

Conectado ao MySQL
Registros inseridos: 36 / 36
Concluído


In [4]:
##Limpa a pasta
import os
dir = '//172.31.8.209/Files/CataliseInvestimentos/14. BigData Catalise/Staging/PosicaoAtivo'
for f in os.listdir(dir):
    os.remove(os.path.join(dir, f))

##OUTROS ATIVOS

In [5]:
import os
import pandas as pd

# Caminho do diretório onde os arquivos estão localizados
caminho_diretorio = '//172.31.8.209/Files/CataliseInvestimentos/14. BigData Catalise/Staging/OutrosAtivos/'

# Lista todos os arquivos no diretório
arquivos = os.listdir(caminho_diretorio)

# Filtra apenas os arquivos CSV
arquivos_csv = [f for f in arquivos if f.endswith('.csv')]

if arquivos_csv:
    # Encontra o arquivo mais recente
    caminho_arquivo_csv = max([os.path.join(caminho_diretorio, f) for f in arquivos_csv], key=os.path.getmtime)

    # Lê o arquivo CSV mais recente
    df_csv_agis = pd.read_csv(caminho_arquivo_csv, delimiter=';', encoding='latin-1', dtype='str', index_col=False)
    
    # Seleciona apenas as colunas desejadas
    colunas_desejadas = ["CARTEIRA", "DATA", "ATIVO", "VALOR"]
    Df_DM = df_csv_agis[colunas_desejadas]

    # Renomeia as colunas
    Df_DM.rename(columns={
        'CARTEIRA': 'NmFundo',
        'DATA': 'DtPosicao',
        'ATIVO': 'Ativo',
        'VALOR': 'VlrAtivo'
    }, inplace=True)

    # Converte a coluna DtPosicao para o formato datetime
    Df_DM['DtPosicao'] = pd.to_datetime(Df_DM['DtPosicao'], format='%d/%m/%Y', errors='coerce')

    # Remover pontos e substituir vírgulas por ponto
    Df_DM['VlrAtivo'] = Df_DM['VlrAtivo'].str.replace('.', '', regex=False)  # Remove pontos
    Df_DM['VlrAtivo'] = Df_DM['VlrAtivo'].str.replace(',', '.', regex=False)  # Substitui vírgula por ponto

    # Converte a coluna VlrAtivo para float
    #Df_DM['VlrAtivo'] = pd.to_numeric(Df_DM['VlrAtivo'], errors='coerce')

    # Formata a coluna VlrAtivo para o formato desejado (com ponto para milhares e vírgula para decimais)
    #Df_DM['VlrAtivo'] = Df_DM['VlrAtivo'].apply(lambda x: '{:,.2f}'.format(x).replace(',', ' ').replace('.', ','))

else:
    print("Nenhum arquivo CSV encontrado no diretório.")

# Exibe o DataFrame
Df_DM.head()


Nenhum arquivo CSV encontrado no diretório.


,NmFundo,DtPosicao,VlrReceber,VlrCaixaAtual,VlrPosicaoAtivo
0,FICFIM STAND BY,2025-03-20,-2210.53,2363.74,1.402270e+06
1,FICFIM AXXIA,2025-03-20,-9580.68,345.13,1.340230e+08
2,FICFIM BINVEST,2025-03-20,-5142.68,2908.69,1.786907e+07
3,FICFIM BELLAGIO,2025-03-20,-3157.90,1489.45,7.838899e+08
4,FIDC BINVEST SR,2025-03-20,0.00,0.00,4.467313e+07


In [6]:
import os
import pandas as pd
import mysql.connector

def AlimentaDM_Ativos(df):
    # Função para inserir um pedaço do DataFrame na tabela MySQL
    def insert_dataframe_to_mysql(df, connection, table_name):
        cursor = connection.cursor()
        for i, row in df.iterrows():
            # Substitui valores NaN por None para que MySQL aceite como NULL
            row = row.where(pd.notna(row), None)
            sql = f"INSERT INTO {table_name} ({', '.join(row.index)}) VALUES ({', '.join(['%s'] * len(row))})"
            cursor.execute(sql, tuple(row))
        connection.commit()
        cursor.close()

    # Conectar ao banco de dados MySQL
    connection = mysql.connector.connect(
        host='catalise-bi-dados-cluster.cluster-ciaao5zm9beh.sa-east-1.rds.amazonaws.com',
        database='DW_CORPORATIVO',
        user='admin',
        password='yb222KraY7PTN0jbH7P3'
    )

    if connection.is_connected():
        print('Conectado ao MySQL')

        # Dividir o DataFrame em pedaços menores para evitar problemas de memória
        tamanho_do_pedaço = 10000  # Ajuste conforme necessário
        total_registros = len(df)

        # Substitui NaN por None nas colunas do DataFrame
        df = df.where(pd.notna(df), None)  # Substitui todos os NaN do DataFrame por None

        for i in range(0, total_registros, tamanho_do_pedaço):
            pedaco_df = df.iloc[i:i + tamanho_do_pedaço]
            insert_dataframe_to_mysql(pedaco_df, connection, 'Dm_OutrosAtivos_LEGADO')
            print(f"Registros inseridos: {i + len(pedaco_df)} / {total_registros}")

        print('Concluído')
    else:
        print('Falha na conexão ao MySQL')

    # Fechar a conexão com o banco de dados
    connection.close()

In [7]:
AlimentaDM_Ativos(Df_DM)

Conectado ao MySQL


ProgrammingError: 1054 (42S22): Unknown column 'VlrReceber' in 'field list'

In [8]:
##Limpa a pasta
import os
dir = '//172.31.8.209/Files/CataliseInvestimentos/14. BigData Catalise/Staging/OutrosAtivos'
for f in os.listdir(dir):
    os.remove(os.path.join(dir, f))